In [1]:
from graphviz import Graph
import numpy as np
from ortools.linear_solver import pywraplp

# 3 node levels: A, B, C
# intra-class weights
A = {
    (0, 1): 5,
    (0, 2): -5,
    (1, 2): -5
}

B = {
    (0, 1): 5,
    (0, 2): 5,
    (1, 2): 5
}

C = {
    (0, 1): 0
}

# inter-class weights
W1 = {
    (0, 0): 5,
    (0, 1): 5,
    (0, 2): -5,
    (1, 0): 5,
    (1, 1): 5,
    (1,2): -5,
    (2,0): -5,
    (2,1):-5,
    (2, 2): 5
}

W2 = {
    (0, 0): 5,
    (0, 1): 5,
    (1, 0): -5,
    (1, 1): 5,
    (2, 0): -5,
    (2, 1): 5
}

get = lambda M, i, j: M[(min(i,j), max(i,j))]
shape = lambda M: (np.max(np.array(list(M.keys()))[:,0]) + 1, \
                np.max(np.array(list(M.keys()))[:,1]) + 1)

Limb_weights = [W1, W2]
Limbs = [(0, 1), (1, 2)]
Joints = [A, B, C]


solver = pywraplp.Solver('SolveAssignmentProblemMIP',
                               pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

Lambda = {}
Iota = {}
Iota_handled = set()
Sums = []

for lid, (W, (a,b)) in enumerate(zip(Limb_weights, Limbs)):
    
    lm_pairs = []
    io1_pairs = []
    io2_pairs = []
    J_a = Joints[a]
    J_b = Joints[b]
    
    n,m = shape(W)
    lm = {}
    for i in range(n):
        for j in range(m):
            Lambda[lid,i,j] = solver.BoolVar('lm%i[%i,%i]' % (lid,i,j))
            lm_pairs.append((i,j))
    
    is_a_handled = a in Iota_handled
    if not is_a_handled:
        Iota_handled.add(a)
        for i in range(n):
            for j in range(n):
                if i == j:
                    continue
                Iota[a,i,j] = solver.BoolVar('iota%i[%i,%i]' % (a,i,j))
                io1_pairs.append((i,j))
                    
    is_b_handled = b in Iota_handled
    if not is_b_handled:
        Iota_handled.add(b)
        for i in range(m):
            for j in range(m):
                if i == j:
                    continue
                Iota[b,i,j] = solver.BoolVar('iota%i[%i,%i]' % (b,i,j))
                io2_pairs.append((i,j))
    
    for i in range(n):
        for j in range(n):
            for k in range(m):
                q1 = (lid, i, k) in Lambda
                q2 = (lid, j, k) in Lambda
                q3 = (a, i, j) in Iota
                if q1 and q2 and q3:
                    solver.Add(Lambda[lid, i, k] + Lambda[lid, j, k] - 1 <= Iota[a, i, j])
                    solver.Add(Lambda[lid, i, k] + Iota[a, i, j] - 1 <= Lambda[lid, j, k])
    
    for i in range(m):
        for j in range(m):
            for k in range(n):
                q1 = (lid, k, i) in Lambda
                q2 = (lid, k, j) in Lambda
                q3 = (b, i, j) in Iota

                if q1 and q2 and q3:
                    solver.Add(Lambda[lid, k, i] + Lambda[lid, k, j] - 1 <= Iota[b, i, j])
                    solver.Add(Lambda[lid, k, i] + Iota[b, i, j] - 1 <= Lambda[lid, k, j])

    sum_lambda = solver.Sum(
            Lambda[lid, i,j] * W[i,j] for i,j in lm_pairs)
    Sums.append(sum_lambda)
    
    if not is_a_handled:
            sum_A = solver.Sum(
                Iota[a,i,j] * get(J_a,i,j) for i,j in io1_pairs)
            Sums.append(sum_A)
    
    if not is_b_handled:
            sum_B = solver.Sum(
                Iota[b,i,j] * get(J_b,i,j) for i,j in io2_pairs)
            Sums.append(sum_B)

solver.Maximize(sum(Sums))
solver.Solve()
print("Time = ", solver.WallTime(), " ms")
                    
                    
print('--')

Time =  12  ms
--


In [2]:
g = Graph('G', engine='dot')


Jname = ['A', 'B', 'C']
Y_pos = [10, 5, 0]
Y_drift = [1, 0, 1]
drawn_joints = set()

for W, (a,b) in zip(Limb_weights, Limbs):
    J_a = Joints[a]
    J_b = Joints[b]
    
    for jid, Jj in zip([a,b], [J_a, J_b]):
        if jid not in drawn_joints:
            drawn_joints.add(jid)
            for (i, j), value in Jj.items():
                g.node(Jname[jid] + str(i), pos=str(i*2) + ',' + str(Y_pos[jid]+Y_drift[i]) + "!")
                g.node(Jname[jid] + str(j), pos=str(j*2) + ',' + str(Y_pos[jid]+Y_drift[j]) + "!")
               
                if value > 0:
                    g.edge(Jname[jid] + str(i), Jname[jid] + str(j),
                          penwidth=str(value),color='#339966')
                elif value < 0:
                    g.edge(Jname[jid] + str(i), Jname[jid] + str(j),
                          penwidth=str(-value),color='#ff6600')
                else:
                    g.edge(Jname[jid] + str(i), Jname[jid] + str(j),
                          penwidth="1",color='#666699')
    
    for (i, j), value in W.items():
        if value > 0:
            g.edge(Jname[a] + str(i), Jname[b] + str(j),
                  penwidth=str(value), color='#99cc00')
        else:
            g.edge(Jname[a] + str(i), Jname[b] + str(j),
                  penwidth=str(-value), color='#ff5050')
    

g.save('wgraph.dot')

'wgraph.dot'

In [3]:
# dot -Kfdp -n -Tpng -o graph.png wgraph.dot && dot -Kfdp -n -Tpng -o egraph.png egraph.dot

g = Graph('G', engine='dot')


Jname = ['A', 'B', 'C']
Y_pos = [10, 5, 0]
Y_drift = [1, 0, 1]
drawn_joints = set()

for lid, (W, (a,b)) in enumerate(zip(Limb_weights, Limbs)):
    J_a = Joints[a]
    J_b = Joints[b]
    
    for jid, Jj in zip([a,b], [J_a, J_b]):
        if jid not in drawn_joints:
            drawn_joints.add(jid)
            for (i, j), value in Jj.items():
                g.node(Jname[jid] + str(i), pos=str(i*2) + ',' + str(Y_pos[jid]+Y_drift[i]) + "!")
                g.node(Jname[jid] + str(j), pos=str(j*2) + ',' + str(Y_pos[jid]+Y_drift[j]) + "!")
                
                if Iota[jid, i, j].solution_value() > 0:
                    g.edge(Jname[jid] + str(i), Jname[jid] + str(j),
                          color='#666699')
            
    
    n, m = shape(W)
    for i in range(n):
        for j in range(m):
            if (lid, i, j) in Lambda:
                if Lambda[lid, i, j].solution_value() > 0:
                    g.edge(Jname[a] + str(i), Jname[b] + str(j))
    
    
    
    #Lambda
            

g.save('egraph.dot')

'egraph.dot'